In [1]:
import pandas as pd
from nltk import tokenize

Загрузим датасет с классификацией записей в сети Твиттер и предполагемой тональностью их содержимого:

In [2]:
df_class = pd.read_csv('twitter_validation.csv', sep=",")
df_class.head()

,ID,Entity,Sentiment,Content
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [11]:
# выделим тестовое сообщение, с которым затем будем выполнять задачи предобработки текста
test_val = 422
texts = df_class['Content']
test_text = texts.iloc[test_val]
test_text

'I’ll never understand how people enjoy overwatch'

# Предобработка текста

## Токенизация

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Токенизация по предложениям:

In [24]:
nltk_tk_sents = nltk.tokenize.sent_tokenize(test_text)
print(len(nltk_tk_sents))
nltk_tk_sents

1


['I’ll never understand how people enjoy overwatch']

Токенизация по словам: 

In [25]:
nltk_tk_1 = nltk.WordPunctTokenizer()
nltk_tk_1.tokenize(test_text)

['I', '’', 'll', 'never', 'understand', 'how', 'people', 'enjoy', 'overwatch']

## Частеречная разметка

In [26]:
from spacy.lang.en import English
import spacy
nlp = spacy.load('en_core_web_sm')
spacy_test = nlp(test_text)

Просмотрим какие части речи присутсвуют в тестовом твите:

In [27]:
for token in spacy_test:
    print('{} - {} - {}'.format(token.text, token.pos_, token.dep_))

I - PRON - nsubj
’ll - VERB - aux
never - ADV - neg
understand - VERB - ROOT
how - ADV - advmod
people - NOUN - nsubj
enjoy - VERB - ccomp
overwatch - PROPN - dobj


## Лемматизация

In [34]:
for token in spacy_test:
      print(token, token.lemma, token.lemma_)

I 561228191312463089 -PRON-
’ll 18307573501153647118 will
never 10879546028211794123 never
understand 2256871070683278179 understand
how 16331095434822636218 how
people 7593739049417968140 people
enjoy 13716726989081948958 enjoy
overwatch 11472399300225631957 overwatch


## Выделение (распознавание) именованных сущностей

In [29]:
for ent in spacy_test.ents:
    print(ent.text, ent.label_)

In [78]:
print(spacy.explain("PEOPLE"))

None


In [31]:
print(spacy.explain("PRODUCT"))

Objects, vehicles, foods, etc. (not services)


In [32]:
from spacy import displacy
displacy.render(spacy_test, style='ent', jupyter=True)

/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


## Разбор предложения

In [33]:
displacy.render(spacy_test, style='dep', jupyter=True)

# Решение задачи классификации текста

In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

Зададим целевую переменную -- тональность текста:

In [ ]:
target = df_class['Sentiment'].values
target

## Способ 1. CountVectorizer

In [40]:
countv = CountVectorizer()
countv_features = countv.fit_transform(df_class["Content"])
countv_features

<1000x5440 sparse matrix of type '<class 'numpy.int64'>'
	with 19225 stored elements in Compressed Sparse Row format>

In [41]:
%%time
score_count_svc = cross_val_score(LinearSVC(), countv_features, target, scoring='accuracy', cv=3).mean()

print('Модель векторизации - Countvectorizer, \nМодель классификации - LinearSVC, \nЗначение accuracy = {}'.format(score_count_svc))

Модель векторизации - Countvectorizer, 
Модель классификации - LinearSVC, 
Значение accuracy = 0.4589919260577943
CPU times: user 46.4 ms, sys: 1.9 ms, total: 48.3 ms
Wall time: 50.9 ms


Получаем достаточно плохой результат. Возможно это следствие особенности текстов в твиттере с ограничением на количество символов, что приводит к сильным сокращениям и искажениям слов. Вообще лексика неформального общения не совсем совпадает со стандартной.

## Способ 2. word2vec

In [42]:
import gensim
from gensim.models import word2vec

In [43]:
import re
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [45]:
# Подготовим корпус
corpus = []
stop_words = stopwords.words('english')
tok = WordPunctTokenizer()
for line in df_class['Content'].values:
    line1 = line.strip().lower()
    line1 = re.sub("[^a-zA-Z]"," ", line1)
    text_tok = tok.tokenize(line1)
    text_tok1 = [w for w in text_tok if not w in stop_words]
    corpus.append(text_tok1)

In [46]:
corpus[:5]

[['mentioned',
  'facebook',
  'struggling',
  'motivation',
  'go',
  'run',
  'day',
  'translated',
  'tom',
  'great',
  'auntie',
  'hayley',
  'get',
  'bed',
  'told',
  'grandma',
  'thinks',
  'lazy',
  'terrible',
  'person'],
 ['bbc',
  'news',
  'amazon',
  'boss',
  'jeff',
  'bezos',
  'rejects',
  'claims',
  'company',
  'acted',
  'like',
  'drug',
  'dealer',
  'bbc',
  'co',
  'uk',
  'news',
  'av',
  'busine'],
 ['microsoft',
  'pay',
  'word',
  'functions',
  'poorly',
  'samsungus',
  'chromebook'],
 ['csgo',
  'matchmaking',
  'full',
  'closet',
  'hacking',
  'truly',
  'awful',
  'game'],
 ['president',
  'slapping',
  'americans',
  'face',
  'really',
  'commit',
  'unlawful',
  'act',
  'acquittal',
  'discover',
  'google',
  'vanityfair',
  'com',
  'news']]

Обучаем модель word2vec на нашем корпусе

In [66]:
%time model_dz = word2vec.Word2Vec(corpus, workers=4, min_count=10, window=10, sample=1e-3)

CPU times: user 83.6 ms, sys: 3.25 ms, total: 86.8 ms
Wall time: 82.6 ms


In [67]:
# Проверим, что модель обучилась
print(model_dz.wv.most_similar(positive=['find'], topn=5))

[('stream', 0.9758626222610474), ('get', 0.9756267070770264), ('com', 0.9746150970458984), ('twitch', 0.974056601524353), ('playing', 0.9735786318778992)]


In [68]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [69]:
class EmbeddingVectorizer(object):
    '''
    Для текста усредним вектора входящих в него слов
    '''
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean(
            [self.model[w] for w in words if w in self.model] 
            or [np.zeros(self.size)], axis=0)
            for words in X])
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [70]:
df_class.shape


(1000, 4)

In [71]:
df_class.head()

,ID,Entity,Sentiment,Content
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [72]:
dz_df = pd.concat([df_class["Content"], df_class["Sentiment"]], axis = 1)

In [73]:
dz_df.head()

,Content,Sentiment
0,I mentioned on Facebook that I was struggling ...,Irrelevant
1,BBC News - Amazon boss Jeff Bezos rejects clai...,Neutral
2,@Microsoft Why do I pay for WORD when it funct...,Negative
3,"CSGO matchmaking is so full of closet hacking,...",Negative
4,Now the President is slapping Americans in the...,Neutral


In [79]:
# Обучающая и тестовая выборки
boundary = 250
X_train = corpus[:boundary] 
X_test = corpus[boundary:]
y_train = dz_df.Sentiment.values[:boundary]
y_test = dz_df.Sentiment.values[boundary:]

In [80]:
%%time
sentiment(EmbeddingVectorizer(model_dz.wv), LogisticRegression(C=5.0))

Метка 	 Accuracy
Irrelevant 	 0.0
Negative 	 0.115
Neutral 	 0.10550458715596331
Positive 	 0.7696078431372549
CPU times: user 50.7 ms, sys: 8.79 ms, total: 59.5 ms
Wall time: 51.9 ms


Результаты, полученные с помощью word2vec тоже не очень хоршие, скорее всего здесь нестандартность лексики ещё больше влияет на работу уже предобученной на более-менее формальных корпусах модели. Короткие неформальные сообщения скорее всего требуют немного других подходов.  